In [1]:
%load_ext autoreload
%autoreload 2

from dataloader import WICO
import tensorflow as tf
from model import Net
from spektral.data import DisjointLoader
from tensorflow import keras
from tqdm import tqdm
import numpy as np

In [6]:
def train_step():
    pass

In [2]:
data = WICO("./dataset/WICO/")
np.random.default_rng(0)

train_pct = 0.2
val_pct = 0.1
num_epochs = 5

len(data)

loader = DisjointLoader(data, batch_size=1, epochs=num_epochs)

In [3]:
model = Net(num_classes=3)

out_log = []

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)


train_loss_results = []
train_accuracy_results = []

epoch_loss_avg = tf.keras.metrics.Mean()
epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

step = 0

for (features, adj, _), y_true in tqdm(loader):
    step += 1
    with tf.GradientTape() as tape:
        # model prediction
        out, _, _ = model([features, adj])            
        y_pred = tf.nn.softmax(out, axis=1)
        loss_val = loss_object(y_true=y_true, y_pred=y_pred)
    
    epoch_loss_avg.update_state(loss_val.numpy())

    epoch_accuracy.update_state(y_true, y_pred)
    grads = tape.gradient(loss_val, model.trainable_variables)

    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    if step % loader.steps_per_epoch == 0:
        epoch = step // loader.steps_per_epoch

        print(f"epoch={epoch}, loss={epoch_loss_avg.result():.3f},"\
               "accuracy={epoch_accuracy.result():.3f}")



79it [00:27,  2.84it/s]


KeyboardInterrupt: 

epoch=4, loss=0.000, accuracy=0.000
